In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Anomaly Detection 

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as py
import seaborn as sns


#warnings
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [ ]:
# load data into the dataframe
df_raw = pd.read_csv("../input/healthcare-providers-data/Healthcare Providers.csv")
df_raw.head()

In [ ]:
# shape of data
df_raw.shape

In [ ]:
df_raw.columns

In [ ]:
DropCols = ['index', 'National Provider Identifier','Last Name/Organization Name of the Provider',
       'First Name of the Provider', 'Middle Initial of the Provider','Street Address 1 of the Provider',
       'Street Address 2 of the Provider','Zip Code of the Provider',"HCPCS Code"]
df_cur1 = df_raw.drop(DropCols, axis = 1)
df_cur1.info()

In [ ]:
100*df_cur1.isnull().sum()/len(df_cur1)

#### We can see that feature 'Credentials of the Provider'  and 'Gender of the Provider' is having less than 10% null values , we can remove these entries or else we can replace them by frequenty appearing values

In [ ]:
print('Frequenty appearing value in "Credentials of the Provider" feature is  ',df_cur1["Credentials of the Provider"].mode()[0])
print('Frequenty appearing value in "Gender of the Provider" feature is  ' ,df_cur1['Gender of the Provider'].mode()[0])

In [ ]:
df_cur1["Credentials of the Provider"] = df_cur1["Credentials of the Provider"].fillna(df_cur1["Credentials of the Provider"].mode()[0])
df_cur1["Gender of the Provider"] = df_cur1["Gender of the Provider"].fillna(df_cur1["Gender of the Provider"].mode()[0])

In [ ]:
# % of null values in dataset
100*df_cur1.isnull().sum()/len(df_cur1)

In [ ]:
df_cur1.describe()

In [ ]:
int_cols = ['Number of Services','Number of Medicare Beneficiaries',
            "Number of Distinct Medicare Beneficiary/Per Day Services",
            'Average Medicare Allowed Amount','Average Submitted Charge Amount',
            'Average Medicare Payment Amount','Average Medicare Standardized Amount']

In [ ]:

df_cur1[int_cols].describe()

In [ ]:
#ValueError: Unable to parse string "1,529.4" at position 122
# In the above error we can see that comma in number, function is created to remove , from that number
# cleanse all the numeric data and convert that table into numeric

def remove_comma(x):
    return x.replace(",","")

for col in int_cols:
    df_cur1[col] = pd.to_numeric(df_cur1[col].apply(lambda x: remove_comma(x)),errors= "ignore")

In [ ]:
df_cur1[int_cols].describe()

#### by seeing STD from the above table , we can say that the data is widley spreaded

In [ ]:
### Lets get categorical columns and there categories

for col in df_cur1.columns:
    if col not in int_cols:
        print(col)
        print(df_cur1[col].value_counts())

In [ ]:
print(int_cols)

In [ ]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
#2.Binary Encoding.

data =   df_cur1  
BEcols = [var for var in data.columns if data[var].dtype == "O"]

for col in BEcols:
    encoder = ce.BinaryEncoder(cols = [col])
    dfbin = encoder.fit_transform(data[col])
    data = pd.concat([data,dfbin], axis = 1)
    del data[col]

In [ ]:
data.head()